In [2]:
from langchain import SQLDatabase
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import os 
# from operator import itemgetter
# from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser

In [3]:
load_dotenv()
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")
DB_URI= os.getenv("DB_URI")
db = SQLDatabase.from_uri(DB_URI)
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

c:\Users\hung\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(


In [4]:

summary_prompt = PromptTemplate(
    input_variables=["chat_history"],
    template="Summarize the following conversation: {chat_history}"
)

# Memory for chat history
memory = ConversationBufferMemory(memory_key="chat_history")

# Function to summarize conversation history
def summarize_history():
    llm_chain = summary_prompt | llm
    summary = llm_chain.invoke({"chat_history":memory.load_memory_variables({})["chat_history"]})
    return summary

# Define tools for the agent to interact with the SQL database
sql_query_prompt = PromptTemplate(
    input_variables=["user_input", "context"],
    template="""
Given an input question, the current context based on the chat history between you and the user and related table schema, generate an SQL query to query a pgvector database that can be used to fetch and return the answer to the question.
Remeber to only output the SQL query (Example: SELECT * FROM "News" WHERE column_name='value')
Rewrite all synonyms and paraphrases of the question into a SQL query that can be used to query the database.
Remember to put the "" around the table name and column name.
Table information:
Name: "news"
Columns and meanings:
- id: The unique identifier for each news article
- title: The title of the news article
- url: The url of the news article
- description: A short description of the news article
- content: The full content of the news article
- cat: The category of the news article
- subcat: The subcategory of the news article
- published_date: The date the news article was published
- author: The author of the news article
- embedding: contains the embeddings of the news article 

Current context: {context}.
Question: {question}
"""
)

sql_generator_chain = sql_query_prompt | llm
sql_executor = QuerySQLDataBaseTool(db=db)

def execute_sql_query(sql_query):
    try:
        # Run the generated SQL query on the database
        result = db.run(sql_query)
        print("SQL RESPONSE: ",result)
        if result is None:
            return "No results found"
        return result
    except Exception as e:
        return f"Error executing query: {e}"
# Define agent with memory for conversation and SQL interaction

result_format_prompt = PromptTemplate(
    input_variables=["sql_result"],
    template="""
You are a highly skilled assistant. The following is the result of a SQL query.
If there are no results, please respond with "No results found".
Result: {sql_result}
Format this result into a user-friendly and readable response.
"""
)

result_format_chain = result_format_prompt | llm

def clean_query(query):
    print("QUERY: ",query)
    return query.replace("```sql", "").replace("```", "").strip()

def get_context():
    return summarize_history()

output_parser=StrOutputParser()

chain = sql_generator_chain | output_parser |  clean_query | execute_sql_query | result_format_chain | output_parser
# Example interaction function
def run_agent(user_input):
    # Get chat history and summarize it
    context = memory.load_memory_variables({}).get("chat_history", "")
    
    # Step 1: Generate SQL query based on user input and context
    response=chain.invoke({"question": user_input, "context": get_context()})
    
    # Save the user input and agent response in memory
    memory.save_context({"input": user_input}, {"response": response})
    
    return response



In [5]:
user_query = "Cho tôi 2 bài báo liên quan đến đại học bách khoa hà nội"
response = run_agent(user_query)
print(response)

QUERY:  SELECT * FROM "news" WHERE "content" ILIKE '%đại học bách khoa hà nội%' LIMIT 2;
SQL RESPONSE:  [(3609, 'Số sinh viên quốc tế đến Việt Nam cao nhất 9 năm', 'https://vnexpress.net/so-sinh-vien-quoc-te-den-viet-nam-cao-nhat-9-nam-4783446.html', 'Gần 22.000 sinh viên nước ngoài đang học tập ở Việt Nam, cao nhất 9 năm qua, theo Bộ Giáo dục và Đào tạo.', 'Trong số này, gần 4.000 người theo diện Hiệp định giữa Việt Nam với các nước, còn lại là diện tự túc chi phí hoặc theo thỏa thuận song phương ở cấp trường và địa phương.\nGiai đoạn 2016-2019, số du học sinh tới Việt Nam không ngừng tăng, từ 17.500 lên 21.000. Tuy nhiên, đến năm 2020-2021, con số...', 'Giáo dục', 'null', '1724130684', 'Thanh Hằng - Dương Tâm', '[-0.036757052,0.04409403,0.020507453,-0.0024506706,0.048664607,-0.009718488,-0.014673954,0.044238362,-0.06384373,-0.004348061,-0.08910217,0.049410332,-0.02884274,0.02961252,0.01569632,0.0048983344,-0.017789163,-0.024284191,0.023442242,-0.03658866,0.00082165125,0.038585283,-0.

In [6]:
def get_current_history():
    history = memory.load_memory_variables({})
    return history.get("chat_history", "No conversation history available.")

# Example usage:
history = get_current_history()
print(history)

Human: Cho tôi 2 bài báo liên quan đến đại học bách khoa hà nội
AI: Here are the latest articles related to education:

1. **Title:** [Số sinh viên quốc tế đến Việt Nam cao nhất 9 năm](https://vnexpress.net/so-sinh-vien-quoc-te-den-viet-nam-cao-nhat-9-nam-4783446.html)  
   **Summary:** Gần 22.000 sinh viên nước ngoài đang học tập ở Việt Nam, cao nhất 9 năm qua, theo Bộ Giáo dục và Đào tạo. Trong số này, gần 4.000 người theo diện Hiệp định giữa Việt Nam với các nước, còn lại là diện tự túc chi phí hoặc theo thỏa thuận song phương ở cấp trường và địa phương. Giai đoạn 2016-2019, số du học sinh tới Việt Nam không ngừng tăng, từ 17.500 lên 21.000. Tuy nhiên, đến năm 2020-2021, con số đã có sự biến động.

   **Author:** Thanh Hằng - Dương Tâm  
   **Published on:** [VnExpress](https://vnexpress.net)  
   **Category:** Giáo dục

2. **Title:** [Tra cứu điểm chuẩn đại học 2024 trên VnExpress](https://vnexpress.net/tra-cuu-diem-chuan-dai-hoc-2024-tren-vnexpress-4781696.html)  
   **Summary:** 